In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key
gmaps.configure(api_key=g_key)

In [4]:
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv", index_col=[0])

In [5]:
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches,Snow Inches
0,Busselton,AU,2020-05-07 16:39:46,-33.65,115.33,57.00,98,54,11.54,light rain,0.23,0.0
1,Hermanus,ZA,2020-05-07 16:39:46,-34.42,19.23,63.00,88,100,3.83,overcast clouds,0.00,0.0
2,Kodiak,US,2020-05-07 16:39:46,57.79,-152.41,44.60,81,90,29.97,light rain,0.25,0.0
3,Lata,PT,2020-05-07 16:39:46,40.16,-8.33,77.00,61,39,5.99,scattered clouds,0.00,0.0
4,Mar Del Plata,AR,2020-05-07 16:36:30,-38.00,-57.56,62.01,59,0,12.75,clear sky,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
569,Pacific Grove,US,2020-05-07 16:41:00,36.62,-121.92,61.00,72,1,6.93,haze,0.00,0.0
570,Francavilla Al Mare,IT,2020-05-07 16:41:00,42.42,14.29,66.20,45,0,6.93,clear sky,0.00,0.0
571,Kavaratti,IN,2020-05-07 16:41:01,10.57,72.64,85.78,71,19,7.76,few clouds,0.00,0.0
572,Suez,EG,2020-05-07 16:36:22,29.97,32.53,79.93,21,0,8.93,clear sky,0.00,0.0


In [6]:
# Count the cities with rain.
city_data_df.loc[city_data_df["Rain Inches"] > 0].count()["City"]

59

In [7]:
# Count the cities with snow.
city_data_df.loc[city_data_df["Snow Inches"] > 0].count()["City"]

5

In [25]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90


In [26]:
# Prompt the customer to answer if he or she would like it to be raining or not
rain_q = input("Do you want it to be raining? (yes/no) ")

Do you want it to be raining? (yes/no) yes


In [27]:
# Prompt the customer to answer if he or she would like it to be snowing or not only if they didn't want it to be raining.
if (rain_q == "no"):
    snow_q = input("Do you want it to be snowing? (yes/no) ")
else:
    snow_q = "no"

In [28]:
if (rain_q.lower() == "no") & (snow_q.lower() == "no"):
    preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                         (city_data_df["Max Temp"] <= max_temp) & 
                                         (city_data_df["Rain Inches"] == 0) &
                                         (city_data_df["Snow Inches"] == 0)]
elif (rain_q.lower() == "no") & (snow_q.lower() == "yes"):
    preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                         (city_data_df["Max Temp"] <= max_temp) &
                                         (city_data_df["Snow Inches"] > 0)]
else:
    preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                         (city_data_df["Max Temp"] <= max_temp) & 
                                         (city_data_df["Rain Inches"] > 0)]
preferred_cities_df.count()["City"]

48

In [29]:
hotel_df=preferred_cities_df[["City","Country","Max Temp","Current Description","Lat","Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

C:\Users\Hannah\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,57.00,light rain,-33.65,115.33,
14,Yima,CN,62.01,moderate rain,34.74,111.88,
37,Rikitea,PF,74.89,light rain,-23.12,-134.97,
40,Bengkulu,ID,78.91,light rain,-3.80,102.27,
66,Hithadhoo,MV,84.51,moderate rain,-0.60,73.08,
94,Vila,PT,78.01,light rain,42.03,-8.16,
103,Biak,ID,78.73,light rain,-0.91,122.88,
109,Kapaa,US,69.80,moderate rain,22.08,-159.32,
124,Noumea,NC,68.00,moderate rain,-22.28,166.46,
127,Kindu,CD,74.41,light rain,-2.95,25.95,


In [30]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [31]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

C:\Users\Hannah\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [32]:
hotel_df.loc[hotel_df["Hotel Name"]!=""].count()

City                   42
Country                42
Max Temp               42
Current Description    42
Lat                    42
Lng                    42
Hotel Name             42
dtype: int64

In [33]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,57.00,light rain,-33.65,115.33,Observatory Guest House
14,Yima,CN,62.01,moderate rain,34.74,111.88,Yima Hotel
37,Rikitea,PF,74.89,light rain,-23.12,-134.97,Pension Maro'i
40,Bengkulu,ID,78.91,light rain,-3.80,102.27,Grage Hotel Bengkulu
66,Hithadhoo,MV,84.51,moderate rain,-0.60,73.08,Scoop Guest House
94,Vila,PT,78.01,light rain,42.03,-8.16,Hotel Castrum Villae
103,Biak,ID,78.73,light rain,-0.91,122.88,Kost Ariz
109,Kapaa,US,69.80,moderate rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
124,Noumea,NC,68.00,moderate rain,-22.28,166.46,Hôtel Le Lagon
127,Kindu,CD,74.41,light rain,-2.95,25.95,Hotel Karibu


In [34]:
# Upload the filtered new DataFrame as WeatherPy_vacation.csv.
hotel_df.to_csv("weather_data/WeatherPy_vacation.csv")

In [35]:
# Add cities to a marker layer map.
locations = hotel_df[["Lat","Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)

fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [37]:
# Add cities to a marker layer map.
locations = hotel_df[["Lat","Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))